In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
# %pip install sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13968\4261329249.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
d:\anaconda\envs\llmapp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = TextLoader("./data/razib.txt")
loaded_document = loader.load()

In [5]:
loaded_document[0].page_content[:1000]

'Razib Dash is an educator and technologist with a strong foundation in computer science, data analytics, and artificial intelligence. With hands-on experience in tools like LangChain, Generative AI, and MERN stack development, Razib blends deep technical skills with a passion for teaching and innovation.\n\nHis areas of interest include:\n\nGenerative AI & Language Models (e.g., LLaMA, ChatGPT)\n\nFull-stack Development (MongoDB, Express, React, Node.js)\n\nData Science & Machine Learning\n\nEducational Technology & AI-driven learning systems\n\nRazib is committed to continuous learning and building intelligent systems that solve real-world problems. Whether in the classroom or the codebase, he values clarity, creativity, and collaboration.\n\nRazib Dash is an academic and researcher specializing in computer science, with particular focus on artificial intelligence, machine learning, and computational education. His work bridges the gap between emerging AI technologies and pedagogical

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=4)
chunks_of_text = text_splitter.split_documents(loaded_document)

Created a chunk of size 305, which is longer than the specified 100
Created a chunk of size 198, which is longer than the specified 100
Created a chunk of size 325, which is longer than the specified 100
Created a chunk of size 374, which is longer than the specified 100


In [7]:
len(chunks_of_text)

12

In [8]:
vector_db = FAISS.from_documents(
    chunks_of_text,
    embeddings,
)
vector_db

In [ ]:
retriever = vector_db.as_retriever()

In [13]:
query = "What is Razib Dash?"
response = retriever.invoke(query)
response

[Document(metadata={'source': './data/razib.txt'}, page_content='Razib Dash is an educator and technologist with a strong foundation in computer science, data analytics, and artificial intelligence. With hands-on experience in tools like LangChain, Generative AI, and MERN stack development, Razib blends deep technical skills with a passion for teaching and innovation.'),
 Document(metadata={'source': './data/razib.txt'}, page_content='Razib Dash is an academic and researcher specializing in computer science, with particular focus on artificial intelligence, machine learning, and computational education. His work bridges the gap between emerging AI technologies and pedagogical innovation, aiming to enhance learning experiences through intelligent systems.'),
 Document(metadata={'source': './data/razib.txt'}, page_content='Razib is committed to continuous learning and building intelligent systems that solve real-world problems. Whether in the classroom or the codebase, he values clarity,

In [15]:
print(response[0].page_content)

Razib Dash is an educator and technologist with a strong foundation in computer science, data analytics, and artificial intelligence. With hands-on experience in tools like LangChain, Generative AI, and MERN stack development, Razib blends deep technical skills with a passion for teaching and innovation.


Simple use with LCEL


In [17]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [22]:
model = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.1,
    max_tokens=512,
    groq_api_key=GROQ_API_KEY,
)


In [23]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [24]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [25]:
response = chain.invoke("what is razib dash?")

In [26]:
response

'Razib Dash is an educator, technologist, academic, and researcher.'

In [27]:
response=chain.invoke("Razib Dash skills and expertise?")
response

"Based on the provided context, Razib Dash's skills and expertise include:\n\n1. Computer science\n2. Data analytics\n3. Artificial intelligence\n4. LangChain\n5. Generative AI\n6. MERN stack development\n7. Machine learning\n8. Computational education\n9. Pedagogical innovation\n10. Building intelligent systems that solve real-world problems."